# Algorithms - Chapter 11 - Balanced Search Trees

## 4/25/2023

### Joy Upton-Azzam

<mark>___________________________________________________________________________________________________________________________</mark>

#### 1. Binary Search Tree

In [1]:
%run "Alg_Ch11_(TreeMap-class).ipynb"

<mark>___________________________________________________________________________________________________________________________</mark>

#### 2. Balanced Trees

The following base class is for balanced trees and includes the relink, rotate, and restructure codes from the text.

source:

https://bcs.wiley.com/he-bcs/Books?action=chapter&bcsId=8029&itemId=1118290275&chapterId=89005

In [2]:
class BalancedTreeMap(TreeMap):
    """A Tree Map Base Class with balancing functions"""
    
    #--------------------- nonpublic methods to support tree balancing ---------------------
    def _relink(self, parent, child, make_left_child):
        """Relink parent node with child node (we allow child to be None)."""
        if make_left_child:                           # make it a left child
            parent.left = child
        else:                                         # make it a right child
            parent.right = child
        if child is not None:                         # make child point to parent
            child.parent = parent

    def _rotate(self, p):
        """Rotate Position p above its parent.

        Switches between these configurations, depending on whether p==a or p==b.

              b                  a
             / \                /  \
            a  t2              t0   b
           / \                     / \
          t0  t1                  t1  t2

        Caller should ensure that p is not the root.
        """
        """Rotate Position p above its parent."""
        x = p.node
        y = x.parent                                 # we assume this exists
        z = y.parent                                 # grandparent (possibly None)
        if z is None:            
            self._root = x                              # x becomes root
            x.parent = None        
        else:
            self._relink(z, x, y == z.left)            # x becomes a direct child of z
        # now rotate x and y, including transfer of middle subtree
        if x == y.left:
            self._relink(y, x.right, True)             # x.right becomes left child of y
            self._relink(x, y, False)                   # y becomes right child of x
        else:
            self._relink(y, x.left, False)             # x.left becomes right child of y
            self._relink(x, y, True)                    # y becomes left child of x

    def _restructure(self, x):
        """Perform a trinode restructure among Position x, its parent, and its grandparent.

        Return the Position that becomes root of the restructured subtree.
    
        Assumes the nodes are in one of the following configurations:
    
            z=a                 z=c           z=a               z=c  
           /  \                /  \          /  \              /  \  
          t0  y=b             y=b  t3       t0   y=c          y=a  t3 
             /  \            /  \               /  \         /  \     
            t1  x=c         x=a  t2            x=b  t3      t0   x=b    
               /  \        /  \               /  \              /  \    
              t2  t3      t0  t1             t1  t2            t1  t2   

        The subtree will be restructured so that the node with key b becomes its root.
    
                  b
                /   \
              a       c
             / \     / \
            t0  t1  t2  t3

        Caller should ensure that x has a grandparent.
        """
        """Perform trinode restructure of Position x with parent/grandparent."""
        y = self.parent(x)
        z = self.parent(y)
        if y and z:
            if (x == self.right(y)) == (y == self.right(z)):    # matching alignments
                self._rotate(y)                                 # single rotation (of y)
                return y                                        # y is new subtree root
            else:                                               # opposite alignments
                self._rotate(x)                                 # double rotation (of x)     
                self._rotate(x)
                return x                                        # x is new subtree root

<mark>___________________________________________________________________________________________________________________________</mark>

#### 3. AVL-Tree

In [3]:
class AVLTreeMap(BalancedTreeMap):
    """Sorted map implementation using an AVL tree."""

    #-------------------------- nested _Node class --------------------------
    class Node(TreeMap.Node):
        """Node class for AVL maintains height value for balancing.

        We use convention that a "None" child has height 0, thus a leaf has height 1.
        """
        __slots__ = '_height'         # additional data member to store height

        def __init__(self, element, parent=None, left=None, right=None):
            super().__init__(element, parent, left, right)
            self._height = 0            # will be recomputed during balancing

        def left_height(self):
            return self.left._height if self.left is not None else 0

        def right_height(self):
            return self.right._height if self.right is not None else 0

    #------------------------- positional-based utility methods -------------------------
    def _recompute_height(self, p):
        p.node._height = 1 + max(p.node.left_height(), p.node.right_height())

    def _isbalanced(self, p):
        return abs(p.node.left_height() - p.node.right_height()) <= 1

    def _tall_child(self, p, favorleft=False): # parameter controls tiebreaker
        if p.node.left_height() + (1 if favorleft else 0) > p.node.right_height():
            return self.left(p)
        else:
            return self.right(p)

    def _tall_grandchild(self, p):
        child = self._tall_child(p)
        # if child is on left, favor left grandchild; else favor right grandchild
        alignment = (child == self.left(p))
        return self._tall_child(child, alignment)

    def _rebalance(self, p):
        while p is not None:
            old_height = p.node._height                          # trivially 0 if new node
            if not self._isbalanced(p):                           # imbalance detected!
                # perform trinode restructuring, setting p to resulting root,
                # and recompute new local heights after the restructuring
                p = self._restructure(self._tall_grandchild(p))
                self._recompute_height(self.left(p))                
                self._recompute_height(self.right(p))                           
            self._recompute_height(p)                             # adjust for recent changes
            if p.node._height == old_height:                     # has height changed?
                p = None                                            # no further changes needed
            else:
                p = self.parent(p)                                  # repeat with parent

  #---------------------------- override balancing hooks ----------------------------
    def _rebalance_insert(self, p):
        self._rebalance(p)

    def _rebalance_delete(self, p):
        self._rebalance(p)

<mark>___________________________________________________________________________________________________________________________</mark>

#### 4. Splay Tree

In [4]:
class SplayTreeMap(BalancedTreeMap):
    """Sorted map implementation using a splay tree."""

    #--------------------------------- splay operation --------------------------------
    def _splay(self, p):
        while p != self.root():
            parent = self.parent(p)
            grand = self.parent(parent)
            if grand is None:
                # zig case
                self._rotate(p)
            elif (parent == self.left(grand)) == (p == self.left(parent)):
                # zig-zig case
                self._rotate(parent)             # move PARENT up
                self._rotate(p)                  # then move p up
            else:
                # zig-zag case
                self._rotate(p)                  # move p up
                self._rotate(p)                  # move p up again

    #---------------------------- override balancing hooks ----------------------------
    def _rebalance_insert(self, p):
        self._splay(p)

    def _rebalance_delete(self, p):
        if p is not None:
            self._splay(p)                     

    def _rebalance_access(self, p):
        self._splay(p)

<mark>___________________________________________________________________________________________________________________________</mark>

#### 5. Red-Black Tree

In [ ]:
class RedBlackTreeMap(BalancedTreeMap):
    """Sorted map implementation using a red-black tree."""

    #-------------------------- nested _Node class --------------------------
    class Node(TreeMap.Node):
        """Node class for red-black tree maintains bit that denotes color."""
        __slots__ = '_red'     # add additional data member to the Node class

        def __init__(self, element, parent=None, left=None, right=None):
            super().__init__(element, parent, left, right)
            self._red = True     # new node red by default

    #------------------------- positional-based utility methods -------------------------
    # we consider a nonexistent child to be trivially black
    def _set_red(self, p): p.node._red = True
    def _set_black(self, p): p.node._red = False
    def _set_color(self, p, make_red): p.node._red = make_red
    def _is_red(self, p): return p is not None and p.node._red
    def _is_red_leaf(self, p): return self._is_red(p) and self.is_leaf(p)

    def _get_red_child(self, p):
        """Return a red child of p (or None if no such child)."""
        for child in (self.left(p), self.right(p)):
            if self._is_red(child):
                return child
        return None
  
    #------------------------- support for insertions -------------------------
    def _rebalance_insert(self, p):
        self._resolve_red(p)                         # new node is always red

    def _resolve_red(self, p):
        if self.is_root(p):
            self._set_black(p)                         # make root black
        else:
            parent = self.parent(p)
            if self._is_red(parent):                   # double red problem
                uncle = self.sibling(parent)
                if not self._is_red(uncle):              # Case 1: misshapen 4-node
                    middle = self._restructure(p)          # do trinode restructuring
                    if middle:
                        self._set_black(middle)                # and then fix colors
                        self._set_red(self.left(middle))
                        self._set_red(self.right(middle))
                else:                                    # Case 2: overfull 5-node
                    grand = self.parent(parent)            
                    if grand:
                        self._set_red(grand)                   # grandparent becomes red
                        self._set_black(self.left(grand))      # its children become black
                        self._set_black(self.right(grand))
                        self._resolve_red(grand)               # recur at red grandparent
      
    #------------------------- support for deletions -------------------------
    def _rebalance_delete(self, p):
        if len(self) == 1:                                     
            self._set_black(self.root())  # special case: ensure that root is black
        elif p is not None:
            n = self.num_children(p)
            if n == 1:                    # deficit exists unless child is a red leaf
                c = next(self.children(p))
                if not self._is_red_leaf(c):
                    self._fix_deficit(p, c)
            elif n == 2:                  # removed black node with red child
                if self._is_red_leaf(self.left(p)):
                    self._set_black(self.left(p))
                else:
                    self._set_black(self.right(p))

    def _fix_deficit(self, z, y):
        """Resolve black deficit at z, where y is the root of z's heavier subtree."""
        if not self._is_red(y): # y is black; will apply Case 1 or 2
            x = self._get_red_child(y)
            if x is not None: # Case 1: y is black and has red child x; do "transfer"
                old_color = self._is_red(z)
                middle = self._restructure(x)
                self._set_color(middle, old_color)   # middle gets old color of z
                self._set_black(self.left(middle))   # children become black
                self._set_black(self.right(middle))
            else: # Case 2: y is black, but no red children; recolor as "fusion"
                self._set_red(y)
                if self._is_red(z):
                    self._set_black(z)                 # this resolves the problem
                elif not self.is_root(z):
                    self._fix_deficit(self.parent(z), self.sibling(z)) # recur upward
        else: # Case 3: y is red; rotate misaligned 3-node and repeat
            self._rotate(y)
            self._set_black(y)
            self._set_red(z)
            if z == self.right(y):
                self._fix_deficit(z, self.left(z))
            else:
                self._fix_deficit(z, self.right(z))